In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install bertopic[visualization]

In [1]:
# Colab może mieć problemy z czytaniem HDFów. Konieczny jest upgrade tables
!pip install --upgrade tables

     |████████████████████████████████| 4.3 MB 7.6 MB/s 
  Attempting uninstall: tables
    Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [2]:
import pandas as pd
import numpy as np
from bertopic import BERTopic

In [ ]:
# Jeżeli bertopic nie chce się zainstalować, należy spróbować najpierw zainstalować hdbscan
# za pomocą pliku WHL z https://www.lfd.uci.edu/~gohlke/pythonlibs/#hdbscan
# bo instalacja hdbscan za pomocą 'pip/conda install' może się nie udać (testowane na Windows 10 + Python 3.7)
# Względnie można nie instalować BERTopic lokalnie i zainstalować go na Colabie bez konfliktów

In [4]:
# Ścieżka do katalogu z notebookami na Colabie
path = '../content/drive/MyDrive/Colab Notebooks/'

In [40]:
df = pd.read_hdf(path+'outputs/budzety_clean.hd5', )

In [54]:
df = df.reset_index()
del df['index']
df

,rok,miasto,nazwa,WSZYSTKO,tokens,zbiornik,sadzić,estetyczny,lęgowy,ekologia,rekultywacja,trawnik,nasadzić,bylina,łąka,park,ogrodniczy,ogród,ozdobny,roślina,kwietny,ekologiczny,zazielenić,parkowy,leśny,natura,las,kwiatowy,roslin,zasadzić,gaj,kwiat,krzew,drzewo,trawiasty,ptasi,sztuczny,zielony,przyroda,przyrodniczy,żywopłot,dębowy,kasztanowiec,leś,powietrze,dziki,naturalny,trawniki,antysmogowy,estetyka,kwietnik,deszczowy,kwitnąć,ogrodowy,pielęgnacja,smog,zieleniec,names,szkola,ulica,oswietlenie,topics
0,2020,GDYNIA,Projektistatus,0,[projektistatus],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,projektistatus,False,False,False,0.0
1,2020,GDYNIA,Doposażenie parku pomiędzy Estakadą Kwiatkowsk...,3,"[doposażenie, park, pomiędzy, estakada, kwiatk...",False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,doposażenie park pomiędzy estakada kwiatkowski...,False,True,False,5.0
2,2020,GDYNIA,Cykliczne warsztaty dla seniorów z dzielnicy O...,0,"[cykliczny, warsztat, senior, dzielnica, obłuż...",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,cykliczny warsztat senior dzielnica obłuże roz...,False,False,False,12.0
3,2020,GDYNIA,"Ogród Sąsiedzki przy Dzikim Sadzie, ul. Źródlana",3,"[ogród, sąsiedzki, dziki, sadź, ul, źródlana]",False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,ogród sąsiedzki dziki sadź ul źródlana,False,True,False,9.0
4,2020,GDYNIA,Montaż słupów oświetleniowych na chodniku przy...,0,"[montaż, słup, oświetleniowy, chodnik, ul, nis...",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,montaż słup oświetleniowy chodnik ul niski odc...,False,True,True,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23025,2020,KRAKÓW,Zielony skwer na Zabtociu,1,"[zielony, skwer, zabtociu]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,zielony skwer zabtociu,False,False,False,NaN
23026,2020,KRAKÓW,Znani Biatopradniczanie dla Mieszkanców,0,"[znany, mieszkanców]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,znany mieszkanców,False,False,

In [55]:
docs = list(df.loc[:, 'names'].values)

docs = [str(elem) for elem in docs] 
# Należy zrzucić docs do stringa, bo inaczej gdzieś w połowie liczenia wyskakuje
# błąd o znalezionych wartościach float:
# sequence item 0: expected str instance, float found

model = BERTopic(language='polish')

In [56]:
# !!!!!! WAŻNE !!!!!!
#
# Jeżeli masz w  folderze models pliki:
# 'main_bert_topic.model' i 'main_bert_topics_probs.npy' to nie uruchamiaj
# tej komórki 
topics, probs = model.fit_transform(docs)
model.save(path+'models/main_bert_topic.model')
with open(path+'models/main_bert_topics_probs.npy', 'wb') as f:
    np.save(f, topics)
    np.save(f, probs)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [66]:
# Wczytanie zapisanych modeli
topic_model = BERTopic.load(path+'models/main_bert_topic.model')
with open(path+'models/main_bert_topics_probs.npy', 'rb') as f:
    topics = np.load(f)
    probs = np.load(f)

In [58]:
# Sprawdzenie tematów
topic_model.get_topics()

{-1: [('jezdny', 0.0014660080439327127),
  ('kreatywny', 0.0014210309396472916),
  ('festyn', 0.0013780181468458206),
  ('hałas', 0.001327446909450214),
  ('kostka', 0.0012947438153774896),
  ('impreza', 0.0012900333045383836),
  ('koncert', 0.0012777641337624024),
  ('rozwój', 0.0012670342975089515),
  ('kierunek', 0.0012614487819886133),
  ('polski', 0.0012511454195671375)],
 0: [('zielono', 0.015427078083611724),
  ('zieleniec', 0.012045192859812072),
  ('zazielenić', 0.008459177193168745),
  ('ściana', 0.007332271023275303),
  ('zakątek', 0.0066590525296619735),
  ('podwórko', 0.006061972537040802),
  ('rekultywacja', 0.0057903578242389335),
  ('torowisko', 0.005681012963732132),
  ('przyuliczny', 0.005160196280696819),
  ('ekologiczny', 0.004277877335074225)],
 1: [('szkolny', 0.01593406852222144),
  ('edukacja', 0.010289877259241652),
  ('dziedziniec', 0.00981015051983764),
  ('korytarz', 0.008343442538440505),
  ('przedszkolny', 0.008129940404459744),
  ('uczeń', 0.0063682790612

In [67]:
# W main_bert_topic.model mamy 371 tematów
# ograniczmy ich ilość
N_TOPICS = 20

In [68]:
# Redukcja tematów 
new_topics, new_probs = topic_model.reduce_topics(docs, topics, probabilities=probs,
 nr_topics=N_TOPICS)

# Zapiszmy nowe tematy
with open(path+'models/new_bert_topics_probs.npy', 'wb') as f:
    np.save(f, new_topics)
    np.save(f, new_probs)

In [69]:
# Sprawdzenie parametrów modelu
topic_model.get_params()

{'calculate_probabilities': False,
 'embedding_model': <bertopic.backend._sentencetransformers.SentenceTransformerBackend at 0x7f4c7de0b110>,
 'hdbscan_model': HDBSCAN(min_cluster_size=10, prediction_data=True),
 'language': 'polish',
 'low_memory': False,
 'min_topic_size': 10,
 'n_gram_range': (1, 1),
 'nr_topics': 20,
 'seed_topic_list': None,
 'top_n_words': 10,
 'umap_model': UMAP(angular_rp_forest=True, low_memory=False, metric='cosine', min_dist=0.0, n_components=5, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True}),
 'vectorizer_model': CountVectorizer(),
 'verbose': False}

In [70]:
# Podgląd tematów
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,7480,-1_chodnik_park_ulica_budowa
1,0,1689,0_parking_dzielnica_rower_parkingowy
2,1,1396,1_fontanna_woda_toaleta_wodny
3,2,989,2_parking_parkingowy_miejsce_bezdomność
4,3,927,3_park_ulica_kieszonkowy_dziecko
5,4,909,4_szkoła_przedszkole_szkolny_przedszkolny
6,5,818,5_biblioteka_książka_audiobook_wydawniczy
7,6,814,6_senior_zdrowie_kobieta_zdrowy
8,7,804,7_chodnik_warszawa_warszawski_warszawo
9,8,770,8_drzewo_ogród_łąka_kwiat


In [71]:
topic_dict = dict()
for n in range(-1, N_TOPICS):
  topic_dict[n] = [name for name, prob in topic_model.get_topic(n)]
  print(f'{n} {topic_dict[n]}', sep='\n')

-1 ['chodnik', 'park', 'ulica', 'budowa', 'warsztat', 'koncert', 'miejsce', 'dzielnica', 'modernizacja', 'muzyczny']
0 ['parking', 'dzielnica', 'rower', 'parkingowy', 'miejski', 'miasto', 'miejsce', 'rowerowy', 'kota', 'ulica']
1 ['fontanna', 'woda', 'toaleta', 'wodny', 'park', 'łazienka', 'kanalizacja', 'rzeka', 'ogród', 'jezioro']
2 ['parking', 'parkingowy', 'miejsce', 'bezdomność', 'szkoła', 'jordanowski', 'językowy', 'miasteczko', 'bezdomny', 'drogowy']
3 ['park', 'ulica', 'kieszonkowy', 'dziecko', 'szach', 'sportowy', 'lodowisko', 'zabawa', 'parking', 'szachowy']
4 ['szkoła', 'przedszkole', 'szkolny', 'przedszkolny', 'edukacyjny', 'młodzież', 'przedszkolak', 'ogród', 'edukacja', 'rekreacyjny']
5 ['biblioteka', 'książka', 'audiobook', 'wydawniczy', 'czytelnik', 'multimedialny', 'czytać', 'książkowy', 'biblioteczny', 'czytanie']
6 ['senior', 'zdrowie', 'kobieta', 'zdrowy', 'seniora', 'kuchnia', 'kulinarny', 'szpital', 'medyczny', 'catering']
7 ['chodnik', 'warszawa', 'warszawski', '

In [72]:
# Wizualizacja
topic_model.visualize_topics()

In [73]:
# Wizualizacja pierwszych 15-stu tematów pod względem word scores
topic_model.visualize_barchart(topics=list(range(N_TOPICS-10)), width=1200, height=900)

In [74]:
# Klastrowanie hierarchiczne
topic_model.visualize_hierarchy(topics=list(range(N_TOPICS)), top_n_topics=10)

In [75]:
# Podobieństwo tematów
topic_model.visualize_heatmap()

In [76]:
# Zbadajmy czy ilość tematów pokrywa się z ilością obserwacji w zbiorze danych
len(new_topics) == df.shape[0]

True

In [77]:
df['topics'] = new_topics

In [85]:
df['topics_desc'] = df['topics'].map(topic_dict)

In [80]:
df['topics'].map(topic_dict)

0        [chodnik, park, ulica, budowa, warsztat, konce...
1        [park, ulica, kieszonkowy, dziecko, szach, spo...
2        [fontanna, woda, toaleta, wodny, park, łazienk...
3        [drzewo, ogród, łąka, kwiat, trawnik, kwiatowy...
4        [chodnik, park, ulica, budowa, warsztat, konce...
                               ...                        
23025    [zielony, zieleń, zielono, ulica, zieleniec, w...
23026    [parking, parkingowy, miejsce, bezdomność, szk...
23027    [powietrze, smog, monitoring, siłownia, szkoła...
23028    [łódź, plaża, plażowy, łódzki, morski, kajakow...
23029    [park, ulica, kieszonkowy, dziecko, szach, spo...
Name: topics, Length: 23030, dtype: object

In [88]:
df

,rok,miasto,nazwa,WSZYSTKO,tokens,zbiornik,sadzić,estetyczny,lęgowy,ekologia,rekultywacja,trawnik,nasadzić,bylina,łąka,park,ogrodniczy,ogród,ozdobny,roślina,kwietny,ekologiczny,zazielenić,parkowy,leśny,natura,las,kwiatowy,roslin,zasadzić,gaj,kwiat,krzew,drzewo,trawiasty,ptasi,sztuczny,zielony,przyroda,przyrodniczy,żywopłot,dębowy,kasztanowiec,leś,powietrze,dziki,naturalny,trawniki,antysmogowy,estetyka,kwietnik,deszczowy,kwitnąć,ogrodowy,pielęgnacja,smog,zieleniec,names,szkola,ulica,oswietlenie,topics,topics_desc
0,2020,GDYNIA,Projektistatus,0,[projektistatus],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,projektistatus,False,False,False,-1,"[chodnik, park, ulica, budowa, warsztat, konce..."
1,2020,GDYNIA,Doposażenie parku pomiędzy Estakadą Kwiatkowsk...,3,"[doposażenie, park, pomiędzy, estakada, kwiatk...",False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,doposażenie park pomiędzy estakada kwiatkowski...,False,True,False,3,"[park, ulica, kieszonkowy, dziecko, szach, spo..."
2,2020,GDYNIA,Cykliczne warsztaty dla seniorów z dzielnicy O...,0,"[cykliczny, warsztat, senior, dzielnica, obłuż...",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,cykliczny warsztat senior dzielnica obłuże roz...,False,False,False,1,"[fontanna, woda, toaleta, wodny, park, łazienk..."
3,2020,GDYNIA,"Ogród Sąsiedzki przy Dzikim Sadzie, ul. Źródlana",3,"[ogród, sąsiedzki, dziki, sadź, ul, źródlana]",False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,ogród sąsiedzki dziki sadź ul źródlana,False,True,False,8,"[drzewo, ogród, łąka, kwiat, trawnik, kwiatowy..."
4,2020,GDYNIA,Montaż słupów oświetleniowych na chodniku przy...,0,"[montaż, słup, oświetleniowy, chodnik, ul, nis...",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,montaż słup oświetleniowy chodnik ul niski odc...,False,True,True,-1,"[chodnik, park, ulica, budowa, warsztat, konce..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23025,2020,KRAKÓW,Zielony skwer na Zabtociu,1,"[zielony, skwer, zabtociu]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,zielony skwer zabtociu,False,False,False,13,"[zielony, zieleń, zielono, ulica, zieleniec, w..."
23026,2020,KRAKÓW,Znani Biatopradniczanie dla Mieszkanców,0,"[znany, mieszkanców]",False,False,False,False,F

In [93]:
# Zapisanie tabeli z przyporządkowanymi tematami do pliku
df.to_hdf(path+'outputs/budzety_clean_topics_bert.h5', key='df')

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2449: PerformanceWarning:


your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['rok', 'miasto', 'nazwa', 'tokens', 'names', 'topics_desc'], dtype='object')]




In [92]:
pd.set_option('max_colwidth', 400)
df[['rok', 'miasto', 'topics', 'topics_desc', 'nazwa', 'names']].sample(10)

,rok,miasto,topics,topics_desc,nazwa,names
7093,2018,POZNAŃ,4,"[szkoła, przedszkole, szkolny, przedszkolny, edukacyjny, młodzież, przedszkolak, ogród, edukacja, rekreacyjny]",Szkoły na glanc,szkoła glanc
2031,2019,SOPOT,19,"[ulica, naprawa, stacja, rower, drogowy, skrzyżowanie, dojazdowy, bezpieczenstwa, budynek, skrzyzowaniu]",3. Remont nawierzchni pasa drogowego i chodników ul. Łużyckiej,remonta nawierzchnia pas drogowy chodnik ul łużycki
14922,2020,ŁÓDŹ,15,"[łódź, plaża, plażowy, łódzki, morski, kajakowy, łódka, kajak, łodzianin, pomorski]",Doposażenie placu zabaw żłobka nr 28 w Łodzi przy Al. S. Wyszyńskiego,doposażenie plac zabawa żłobek łódź al wyszyńskiego
18589,2018,WARSZAWA,12,"[pies, psi, ursynów, pszczoła, komar, ursynowie, owad, nietoperz, zwierzę, wiewiórka]",Scrabble na Ursynowie,scrabble ursynowie
6735,2018,KALISZ,9,"[autobusowy, ścieżka, autobus, tramwaj, asfaltowy, ciąg, tramwajowy, ulica, asfalt, przejazd]",KALISZANIE Z NAPĘDEM 4x4. Nauka prawidłowej techniki Nordic Walking dla mieszkańców Kalisza.,kaliszanin napęd nauka prawidłowy technika nordic walking mieszkaniec kalisza
17359,2018,TARNOBRZEG,16,"[siłownia, piknik, rekreacja, rekreacyjny, aktywny, piknikowy, park, zabawa, szkolenie, sportowy]",Siłownia napowietrzna.,siłownia napowietrzny
8419,2020,ŚWINOUJŚCIE,14,"[cmentarz, kościół, pomnik, rewitalizacja, rzeźba, wieża, chodnik, ulica, obserwatorium, widokowy]",PREWENCYJNY MONITORING CMENTARZA KOMUNALNEGO,prewencyjny monitoring cmentarz komunalny
6748,2018,KALISZ,1,"[fontanna, woda, toaleta, wodny, park, łazienka, kanalizacja, rzeka, ogród, jezioro]",Kaliscy podróżnicy – cykl prelekcji.,kaliski podróżnik cykl prelekcja
16796,2020,PŁOCK,16,"[siłownia, piknik, rekreacja, rekreacyjny, aktywny, piknikowy, park, zabawa, szkolenie, sportowy]","Rodzinny, wielofunkcyjny skwer wypoczynku i rekreacji wraz z remontem muszli koncertowej",rodzinny wielofunkcyjny skwer wypoczynek rekreacja wraz remont muszla koncertowy
22952,2020,KRAKÓW,8,"[drzewo, ogród, łąka, kwiat, trawnik, kwiatowy, botaniczny, kwitnąć, żywopłot, ogródka]",tawki parkowe na Plantach Mistrzejowickich,tawki parkowy planty
